In [4]:
from keras.applications import InceptionV3
from keras.models import Model
from keras.layers import Dropout, Input, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


In [5]:
batch_size = 8;

Train_DataSet_Path = r'D:\Facultate\Anul4\Sem1\CHS\Datasets\Eyes_DataSet\Train_Dataset'

train_data_generator = ImageDataGenerator(rotation_range=0.2, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2,zoom_range=0.2, rescale=1./255, validation_split=0.2)
train_data = train_data_generator.flow_from_directory(Train_DataSet_Path,
                                               target_size=(80, 80), batch_size=batch_size, class_mode='categorical',
                                               subset='training')
validation_data = train_data_generator.flow_from_directory(Train_DataSet_Path,
                                                    target_size=(80, 80), batch_size=batch_size, class_mode='categorical',
                                                    subset='validation')

Found 61127 images belonging to 2 classes.
Found 15281 images belonging to 2 classes.


In [6]:
Test_DataSet_Path = r'D:\Facultate\Anul4\Sem1\CHS\Datasets\Eyes_DataSet\Test_Dataset'

test_data_generator = ImageDataGenerator(rescale=1./255)
test_data = test_data_generator.flow_from_directory(Test_DataSet_Path,
                                             target_size=(80, 80), batch_size=batch_size, class_mode='categorical')

Found 8490 images belonging to 2 classes.


In [7]:
base_model = InceptionV3(include_top=False, weights= "imagenet", input_tensor=Input(shape=(80, 80, 3)))
head_model = base_model.output
head_model = Flatten()(head_model)
head_model = Dense(64, activation="relu")(head_model)
head_model = Dropout(0.5)(head_model)
head_model = Dense(2, activation='softmax')(head_model)

final_model =  Model(inputs=base_model.input, outputs=head_model)
for layer in base_model.layers:
    layer.trainable = False


In [8]:
checkpoint = ModelCheckpoint(r'D:\UPT\Anul IV\Sem1\CHS\Proiect\Models\model.h5', monitor='val_loss', save_best_only=True, verbose=3)
earlystop = EarlyStopping(monitor='val_loss', patience=7, verbose=3, restore_best_weights=True)
learning_rate = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=3)
callbacks = [checkpoint, earlystop, learning_rate]



In [9]:
final_model.compile(loss='categorical_crossentropy', optimizer='Adam',  metrics=['accuracy'])
final_model.fit(train_data, steps_per_epoch=train_data.samples//batch_size, validation_data=validation_data, validation_steps=validation_data.samples//batch_size, callbacks=callbacks, epochs=5)  #8 is the batch_size

Epoch 1/5
7640/7640 [==============================] - ETA: 0s - loss: 0.2116 - accuracy: 0.9170
Epoch 1: val_loss improved from inf to 0.21604, saving model to D:\UPT\Anul IV\Sem1\CHS\Proiect\Models\model.h5
7640/7640 [==============================] - 1087s 142ms/step - loss: 0.2116 - accuracy: 0.9170 - val_loss: 0.2160 - val_accuracy: 0.9124 - lr: 0.0010
Epoch 2/5
7638/7640 [============================>.] - ETA: 0s - loss: 0.1798 - accuracy: 0.9318
Epoch 2: val_loss improved from 0.21604 to 0.20235, saving model to D:\UPT\Anul IV\Sem1\CHS\Proiect\Models\model.h5
7640/7640 [==============================] - 234s 31ms/step - loss: 0.1798 - accuracy: 0.9318 - val_loss: 0.2024 - val_accuracy: 0.9111 - lr: 0.0010
Epoch 3/5
7640/7640 [==============================] - ETA: 0s - loss: 0.1731 - accuracy: 0.9341
Epoch 3: val_loss improved from 0.20235 to 0.18114, saving model to D:\UPT\Anul IV\Sem1\CHS\Proiect\Models\model.h5
7640/7640 [==============================] - 241s 32ms/step - los

In [14]:
print(final_model.evaluate(train_data))
print(final_model.evaluate(validation_data))
print(final_model.evaluate(test_data))

7641/7641 [==============================] - 197s 26ms/step - loss: 0.1388 - accuracy: 0.9456
[0.13883282244205475, 0.9455723166465759]
1911/1911 [==============================] - 46s 24ms/step - loss: 0.2090 - accuracy: 0.9172
[0.20902425050735474, 0.9172174334526062]
1062/1062 [==============================] - 26s 25ms/step - loss: 0.3270 - accuracy: 0.9000
[0.32696133852005005, 0.8999999761581421]
